In [3]:
from bing_image_downloader import downloader

# Donwload images with cats
downloader.download(
    "cat",
    limit=500,
    output_dir='cats-dataset',
       )
# Donwload images with dogs
downloader.download(
    "dog",
    limit=500,
    output_dir='dogs-dataset',
       )


[%] Downloading Images to /home/nykfer/projects/nn/classification/cats-dataset/cat


[!!]Indexing page: 1

[%] Indexed 65 Images on Page 1.


[%] Downloading Image #1 from http://assets.stickpng.com/thumbs/580b57fbd9996e24bc43bb91.png
[%] File Downloaded !

[%] Downloading Image #2 from https://forum.lowyat.net/uploads/attach-92/post-180892-1384103512_thumb.jpg
[%] File Downloaded !

[%] Downloading Image #3 from https://forgifs.com/gallery/d/276122-2/RoboCat-staring-cat-reaction.gif
[!] Issue getting: https://forgifs.com/gallery/d/276122-2/RoboCat-staring-cat-reaction.gif
[!] Error:: HTTP Error 404: Not Found
[%] Downloading Image #3 from https://www.bing.com/th/id/OGC.15ff66a909143b0f85a04e509b85081c?o=7&amp;pid=1.7&amp;rm=3&amp;rurl=https%3a%2f%2fforgifs.com%2fgallery%2fd%2f276122-2%2fRoboCat-staring-cat-reaction.gif&amp;ehk=ky76L2GWAn8B9PZIALT8oJ9eZbShb6YvVpKA2THyrwQ%3d
[!] Issue getting: https://www.bing.com/th/id/OGC.15ff66a909143b0f85a04e509b85081c?o=7&amp;pid=1.7&amp;rm=3&amp;r

In [7]:
import os
from PIL import Image

def count_images_in_folder(folder_name):
    folder_path = os.path.join(os.getcwd(), folder_name)
    count = 0
    for name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, name)
        if os.path.isfile(file_path):
            try:
                with Image.open(file_path) as img:
                    img.verify()  # verify it's an image
                count += 1
            except Exception:
                pass  # not an image file
    return count

cat_images_count = count_images_in_folder('cats-dataset/cat')
dog_images_count = count_images_in_folder('dogs-dataset/dog')
print(f"Number of images in cats-dataset: {cat_images_count}")
print(f"Number of images in dogs-dataset: {dog_images_count}")

Number of images in cats-dataset: 90
Number of images in dogs-dataset: 84


In [8]:
import pandas as pd
from PIL import Image

def get_image_info(folder_name):
    folder_path = os.path.join(os.getcwd(), folder_name)
    file_records = []
    for name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, name)
        if os.path.isfile(file_path):
            try:
                with Image.open(file_path) as img:
                    width, height = img.size
                    img_format = img.format
            except Exception as e:
                width, height, img_format = None, None, None  # For non-image or corrupted files
            file_stats = os.stat(file_path)
            file_records.append({
                'file_name': name,
                'folder': folder_name,
                'size_bytes': file_stats.st_size,
                'image_width': width,
                'image_height': height,
                'format': img_format
            })
    return file_records

# Get info for both datasets
cats_info = get_image_info('cats-dataset/cat')
dogs_info = get_image_info('dogs-dataset/dog')

# Combine info and create a DataFrame
all_info = cats_info + dogs_info
df = pd.DataFrame(all_info)

# Add an image count by group (folder)
df['image_count_in_folder'] = df.groupby('folder')['file_name'].transform('count')

# Show the DataFrame
print(df.head())
# Optionally, save the df to a CSV
df.to_csv('image_file_properties.csv', index=False)



       file_name            folder  size_bytes  image_width  image_height  \
0   Image_45.jpg  cats-dataset/cat      111945          478           720   
1   Image_72.jpg  cats-dataset/cat      445977         2560          1696   
2    Image_9.jpg  cats-dataset/cat       82863         1200           800   
3  Image_38.jpeg  cats-dataset/cat      215129         1536          1536   
4    Image_1.png  cats-dataset/cat       64959          400           400   

  format  image_count_in_folder  
0   JPEG                     90  
1   JPEG                     90  
2   JPEG                     90  
3   JPEG                     90  
4    PNG                     90  


In [15]:

table = df.drop(columns=['file_name', 'size_bytes', 'folder', 'image_count_in_folder']) \
    .groupby(['image_height', 'image_width']) \
    .agg({'format': 'count'}) \
    .rename(columns={'format': 'count'}) \
    .sort_values(by='count', ascending=False)
table

,,count
image_height,image_width,
800,1200,8
1200,1600,7
1080,1920,6
768,1024,6
1280,1920,4
...,...,...
3313,4969,1
3505,5257,1
3840,2025,1


In [27]:

df.nsmallest(2, 'image_height'), df.nlargest(2, 'image_width') 

(       file_name            folder  size_bytes  image_width  image_height  \
 63  Image_21.png  cats-dataset/cat       39601          200           200   
 42  Image_26.gif  cats-dataset/cat      641460          220           214   
 
    format  image_count_in_folder  
 63    PNG                     90  
 42    GIF                     90  ,
          file_name            folder  size_bytes  image_width  image_height  \
 110   Image_11.jpg  dogs-dataset/dog     5422563         6000          4000   
 29   Image_42.jpeg  cats-dataset/cat     2347419         5472          3648   
 
     format  image_count_in_folder  
 110   JPEG                     84  
 29    JPEG                     90  )